In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.7 MB/s 
     |████████████████████████████████| 453 kB 54.2 MB/s 


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json
import numpy as np
from konlpy.tag import Okt 
from collections import Counter
import re

# Musinsa Standard _ 상품번호 / 상품설명 크롤링

In [9]:
Explanation = {
              '상품번호': [],
              '상품설명':[]
}


무신사 모든 페이지 (후기 순 기준 총 페이지 : 44 페이지)

상품 번호 크롤링 (44 페이지 까지의 상품번호 개수 : 3926) 


In [13]:
for idx in range(1, 45):
    url = 'https://www.musinsa.com/brands/musinsastandard'

    params = {
                'sortCode':'emt_high',
                'page':str(idx),
                'size':'90',
                'listViewType':'small',
                'outletGoods':'false'
              }


    resp = requests.get(url,params = params)
    soup = BeautifulSoup(resp.content, 'lxml')

# resp

    item_num = soup.select('div.list_img a')
# item_num
    # ['href'].replace('//store.musinsa.com/app/goods/','')
    for num in item_num:
        Explanation['상품번호'].append(num['href'].replace('//www.musinsa.com/app/goods/',''))
Explanation['상품번호']

['1220731',
 '1149328',
 '1149329',
 '996177',
 '1144989',
 '1168906',
 '1108007',
 '803032',
 '996178',
 '991339',
 '1382658',
 '1129745',
 '1222184',
 '606312',
 '746159',
 '1136908',
 '1558197',
 '991340',
 '1504726',
 '763134',
 '1283757',
 '996497',
 '1190264',
 '1224095',
 '2086653',
 '947067',
 '746158',
 '865862',
 '792919',
 '1136909',
 '947088',
 '1212777',
 '1212776',
 '1417691',
 '1094748',
 '1168922',
 '1323968',
 '728000',
 '1190268',
 '1440874',
 '1312035',
 '854337',
 '2034137',
 '2149254',
 '996498',
 '965917',
 '1216295',
 '996234',
 '1324127',
 '1388775',
 '1221497',
 '947202',
 '1249102',
 '996233',
 '1231416',
 '1291189',
 '792920',
 '1034119',
 '735511',
 '1436504',
 '983818',
 '1254403',
 '991341',
 '1323974',
 '1322221',
 '947201',
 '983339',
 '1168907',
 '1388776',
 '1321881',
 '1666442',
 '896527',
 '792921',
 '1642916',
 '1322219',
 '1190282',
 '2035287',
 '746161',
 '1595638',
 '996193',
 '803037',
 '1129742',
 '1417715',
 '1417692',
 '1215265',
 '983819',
 

상품 번호 별 상품 상세설명 크롤링

상세설명이 없는 페이지도 있어 없는 것은 '-'값 처리

In [14]:
for page in tqdm(Explanation['상품번호']):

      url = 'https://store.musinsa.com/app/goods/'+str(page)

      headers = {
                'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
                # 'referer': 'https://www.musinsa.com/'
                'referer': 'https://www.musinsa.com/'
      }
      
      resp = requests.get(url, headers = headers)
      soup = BeautifulSoup(resp.content, 'lxml')
      # resp

      l_tags = soup.select('div.product_left p.box_opinion_product')   

      if l_tags == []:
        Explanation['상품설명'].append('-')
      else:
        Explanation['상품설명'].append(l_tags[0].text)

  0%|          | 0/3944 [00:00<?, ?it/s]

In [15]:
df = pd.DataFrame(Explanation)

In [16]:
df

,상품번호,상품설명
0,1220731,베이식한 디자인의 사피아노 신세틱 레더 벨트.\n포멀 룩부터 캐주얼 룩까지 다양하게...
1,1149328,편안하면서도 슬림한 테이퍼드 핏의 히든 밴딩 크롭 슬랙스.\n4%의 폴리우레탄이 포...
2,1149329,여유 있는 실루엣의 세미 와이드 핏 히든 밴딩 슬랙스.\n4%의 폴리우레탄이 포함된...
3,996177,여유 있는 릴렉스 핏의 크루 넥 티셔츠.\n촘촘하게 짜여진 면 100% 코마사 16...
4,1144989,여유 있는 릴렉스 핏의 베이식 롱 슬리브 티셔츠.\n\n촘촘하게 짜여진 면 100%...
...,...,...
3939,2222923,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3940,2222922,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3941,2222921,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3942,2036928,편안하면서도 슬림한 테이퍼드 핏의 울 히든 밴딩 크롭 슬랙스. 스테디 셀러인 테이퍼...


중복 값 처리 시 같은 키워드 상품 삭제 우려 

중복 처리 하여 같은 항목 삭제 하면 옷 추천 시 추천 할 수 없음 

중복 하는 상세설명끼리 묶을 수는 없나?

In [19]:
import pandas as pd

In [21]:
df.to_csv('total_itemdetail.csv')

In [22]:
df1 = pd.read_csv('/content/total_itemdetail.csv', index_col=0)  

In [23]:
df1

,상품번호,상품설명
0,1220731,베이식한 디자인의 사피아노 신세틱 레더 벨트.\n포멀 룩부터 캐주얼 룩까지 다양하게...
1,1149328,편안하면서도 슬림한 테이퍼드 핏의 히든 밴딩 크롭 슬랙스.\n4%의 폴리우레탄이 포...
2,1149329,여유 있는 실루엣의 세미 와이드 핏 히든 밴딩 슬랙스.\n4%의 폴리우레탄이 포함된...
3,996177,여유 있는 릴렉스 핏의 크루 넥 티셔츠.\n촘촘하게 짜여진 면 100% 코마사 16...
4,1144989,여유 있는 릴렉스 핏의 베이식 롱 슬리브 티셔츠.\n\n촘촘하게 짜여진 면 100%...
...,...,...
3939,2222923,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3940,2222922,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3941,2222921,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷.\n600FP의 덕 다운을 105g...
3942,2036928,편안하면서도 슬림한 테이퍼드 핏의 울 히든 밴딩 크롭 슬랙스. 스테디 셀러인 테이퍼...


# Musinsa Standard 상품 설명 불용어 제거

In [24]:
def apply_regular_expression(text):
    hangul = re.compile('[^ㄱ-ㅣ 가-힣]')
    result = hangul.sub('',text)
    return result

In [25]:
df1['상품설명'] = df1['상품설명'].apply(apply_regular_expression)

In [26]:
df1

,상품번호,상품설명
0,1220731,베이식한 디자인의 사피아노 신세틱 레더 벨트포멀 룩부터 캐주얼 룩까지 다양하게 활용...
1,1149328,편안하면서도 슬림한 테이퍼드 핏의 히든 밴딩 크롭 슬랙스의 폴리우레탄이 포함된 양방...
2,1149329,여유 있는 실루엣의 세미 와이드 핏 히든 밴딩 슬랙스의 폴리우레탄이 포함된 양방향 ...
3,996177,여유 있는 릴렉스 핏의 크루 넥 티셔츠촘촘하게 짜여진 면 코마사 수 싱글 저지 원...
4,1144989,여유 있는 릴렉스 핏의 베이식 롱 슬리브 티셔츠촘촘하게 짜여진 면 코마사 수 싱글...
...,...,...
3939,2222923,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷의 덕 다운을 기준 충전해 적절한...
3940,2222922,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷의 덕 다운을 기준 충전해 적절한...
3941,2222921,여유 있는 레귤러 핏의 칼라리스 다운 집업 재킷의 덕 다운을 기준 충전해 적절한...
3942,2036928,편안하면서도 슬림한 테이퍼드 핏의 울 히든 밴딩 크롭 슬랙스 스테디 셀러인 테이퍼드...


In [27]:
df1.to_csv('rm_stopwords_total_itemdetail.csv')

In [28]:
df2 = pd.read_csv('/content/rm_stopwords_total_itemdetail.csv')

In [29]:
df2_drop = df2.drop_duplicates('상품설명')

In [31]:
df2_drop

,Unnamed: 0,상품번호,상품설명
0,0,1220731,베이식한 디자인의 사피아노 신세틱 레더 벨트포멀 룩부터 캐주얼 룩까지 다양하게 활용...
1,1,1149328,편안하면서도 슬림한 테이퍼드 핏의 히든 밴딩 크롭 슬랙스의 폴리우레탄이 포함된 양방...
2,2,1149329,여유 있는 실루엣의 세미 와이드 핏 히든 밴딩 슬랙스의 폴리우레탄이 포함된 양방향 ...
3,3,996177,여유 있는 릴렉스 핏의 크루 넥 티셔츠촘촘하게 짜여진 면 코마사 수 싱글 저지 원...
4,4,1144989,여유 있는 릴렉스 핏의 베이식 롱 슬리브 티셔츠촘촘하게 짜여진 면 코마사 수 싱글...
...,...,...,...
3882,3882,2653649,미니멀한 디자인의 우먼즈 후디드 다운 푸퍼 숏 패딩 재킷필 파워 의 덕 다운을 사...
3887,3887,2611818,무신사 스탠다드와 유아인이 함께한 년을 기념해 제작한 캐시미어 블렌드 빅 대디 싱글...
3900,3900,2542832,여유로운 실루엣의 캐시미어 블렌드 오버사이즈 싱글 코트어깨선을 자연스럽게 드롭시킨 ...
3903,3903,2533590,미니멀한 디자인의 다운 푸퍼 숏 패딩 재킷필파워 의 덕 다운을 사이즈 기준충전해 ...


In [30]:
okt = Okt()

In [32]:
new_dic = []

In [33]:
for code in tqdm(df['상품설명']):
    new_dic.append(okt.nouns(code))

  0%|          | 0/3944 [00:00<?, ?it/s]

In [35]:
new_dic

[['베이식',
  '디자인',
  '피아노',
  '신세',
  '레더',
  '벨트',
  '포',
  '룩',
  '캐',
  '주얼',
  '룩',
  '활용',
  '수',
  '개발',
  '버클',
  '도금',
  '후',
  '표면',
  '새틴',
  '가공',
  '마무리',
  '움',
  '가죽',
  '스트랩',
  '폭',
  '로',
  '설정',
  '슬랙스',
  '데님',
  '팬츠',
  '착',
  '벨트',
  '겉면',
  '스크래치',
  '피아노',
  '패턴',
  '신세',
  '레더',
  '사용',
  '내부',
  '보강',
  '재',
  '구성',
  '고급',
  '벨트',
  '전문',
  '제작',
  '업체',
  '공장',
  '공정',
  '제작',
  '검품',
  '과정',
  '수준',
  '완성',
  '자랑',
  '취향',
  '위해',
  '버클',
  '컬러',
  '옵션',
  '구성',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리',
  '인치',
  '기준',
  '인치',
  '사이즈',
  '허리